In [6]:
import nltk

In [7]:
import jieba
import tflearn

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [8]:
import tensorflow as tf

# read the preprocressed corpus 

In [9]:
import pickle
with open('middleresult/tokenlizer_output_100000ch_120000en_35words.pkl','rb') as fhdl:
    (
         ind2ch,
         ch2ind,
         ind2en,
         en2ind,
         train_x,
         train_y,
    ) = pickle.load(fhdl)

In [10]:
len(ind2ch),len(ind2en)

(100000, 120000)

# define the model hyperparams

In [11]:
src_vocab_size = 50003#len(ind2en) + 3
target_vocat_size = 50003#len(ind2ch) + 3
attention_hidden_size = 1024
attention_output_size = 1024
embedding_size = 1024
seq_max_len = 40
num_units = 1024
batch_size = 64
layer_number = 4
max_grad = 1.0
dropout = 0.2

In [12]:
len(train_x),len(train_y)

(1797140, 1797140)

# process the model input

In [13]:
train_x = tf.contrib.keras.preprocessing.sequence.pad_sequences(train_x,seq_max_len,padding='post')
train_y = tf.contrib.keras.preprocessing.sequence.pad_sequences(train_y,seq_max_len,padding='post')

In [14]:
import random
index = random.randint(0,len(train_x))
print(' '.join([ind2en.get(i,'') for i in train_x[index]]))
print(' '.join([ind2ch.get(i,'') for i in train_y[index]]))

the hunter watched for the hare to come out of the burrow .                           
猎人 守候 着 兔子 从 洞里 出来 。                                


In [15]:
train_x.shape,train_y.shape

((1797140, 40), (1797140, 40))

In [16]:
from sklearn.cross_validation import train_test_split

In [17]:
train_x,test_x,train_y,test_y = train_test_split(train_x,train_y , test_size=0.01, random_state=42)

In [18]:
train_x[train_x >= src_vocab_size] = 1 
test_x[test_x >= src_vocab_size] = 1 

In [19]:
train_y[train_y >= target_vocat_size] = 1 
test_y[test_y >= target_vocat_size] = 1 

In [20]:
import numpy as np
x_last_index = src_vocab_size - 1
y_last_index = src_vocab_size - 1

ind2en[x_last_index] = '<go>'
ind2ch[y_last_index] = '<go>'
en2ind['<go>'] = x_last_index
ch2ind['<go>'] = y_last_index

ind2en[1] = '<unk>'
ind2ch[1] = '<unk>'
en2ind['<unk>'] = 1
ch2ind['<unk>'] = 1

In [22]:
import random
for i in range(5):
    index = random.randint(0,len(train_x))
    print(' '.join([ind2en.get(i,'') for i in train_x[index]]))
    print(' '.join([ind2ch.get(i,'') for i in train_y[index]]))

being a modest person , he was content to play second fiddle to others .                         
作为 一个 谦虚 者 ， 他 愿意 居于 他人 的 次要 位置 。                           
and dad says , " <unk> , give <unk> more space around the dinner table . "                       
我爸 说 ， <unk> ， 让 点 位子 给 盖尔                              
and tonight is our night , so relax .                               
今晚 是 我们 的 时刻 ， 放松 吧 ！                               
the key to solving the problems we are now facing                              
解决 我们 面临 问题 的 关键                                  
the book says cut the red wire .                                
书上 说 要 剪 红线                                   


# define the translate nmt model

In [23]:
from tensorflow.python.layers import core as layers_core

In [24]:
import tensorflow as tf
import tflearn
tf.reset_default_graph()
config = tf.ConfigProto(log_device_placement=True,allow_soft_placement = True)
config.gpu_options.allow_growth = True
#config.gpu_options.per_process_gpu_memory_fraction = 0.4
session = tf.Session(config=config)


with tf.device('/gpu:0'):
    initializer = tf.random_uniform_initializer(
        -0.08, 0.08)
    tf.get_variable_scope().set_initializer(initializer)
    
    x = tf.placeholder("int32", [None, None])
    y = tf.placeholder("int32", [None, None])
    y_in = tf.placeholder("int32",[None,None])
    x_len = tf.placeholder("int32",[None])
    y_len = tf.placeholder("int32",[None])
    x_real_len = tf.placeholder("int32",[None])
    y_real_len = tf.placeholder("int32",[None])
    learning_rate = tf.placeholder(tf.float32, shape=[])
    
    # embedding
    embedding_encoder = tf.get_variable(
        "embedding_encoder", [src_vocab_size, embedding_size],dtype=tf.float32)
    embedding_decoder = tf.get_variable(
        "embedding_decoder", [target_vocat_size, embedding_size],dtype=tf.float32)
    
    encoder_emb_inp = tf.nn.embedding_lookup(
        embedding_encoder, x)
    decoder_emb_inp = tf.nn.embedding_lookup(
        embedding_decoder, y_in)
    
    # encoder
    num_bi_layers = int(layer_number / 2)
    cell_list = []
    for i in range(num_bi_layers):
        cell_list.append(
            tf.contrib.rnn.DropoutWrapper(
                tf.contrib.rnn.BasicLSTMCell(num_units), input_keep_prob=(1.0 - dropout)
            )
        )
    if len(cell_list) == 1:
        encoder_cell = cell_list[0]
    else:
        encoder_cell = tf.contrib.rnn.MultiRNNCell(cell_list)
        
    cell_list = []
    
    for i in range(num_bi_layers):
        cell_list.append(
            tf.contrib.rnn.DropoutWrapper(
                tf.contrib.rnn.BasicLSTMCell(num_units), input_keep_prob=(1.0 - dropout)
            )
        )
    if len(cell_list) == 1:
        encoder_backword_cell = cell_list[0]
    else:
        encoder_backword_cell = tf.contrib.rnn.MultiRNNCell(cell_list)
    
    bi_outputs, bi_encoder_state = tf.nn.bidirectional_dynamic_rnn(
        encoder_cell,encoder_backword_cell, encoder_emb_inp,
        sequence_length=x_len,dtype=tf.float32)
    encoder_outputs = tf.concat(bi_outputs, -1)
    
    if num_bi_layers == 1:
        encoder_state = bi_encoder_state
    else:
        encoder_state = []
        for layer_id in range(num_bi_layers):
            encoder_state.append(bi_encoder_state[0][layer_id])  # forward
            encoder_state.append(bi_encoder_state[1][layer_id])  # backward
        encoder_state = tuple(encoder_state)
    
    # decoder 
    #decoder_cell = tf.contrib.rnn.BasicLSTMCell(num_units)
    cell_list = []
    for i in range(layer_number):
        cell_list.append(
            tf.contrib.rnn.DropoutWrapper(
                tf.contrib.rnn.BasicLSTMCell(num_units), input_keep_prob=(1.0 - dropout)
            )
        )
    if len(cell_list) == 1:
        decoder_cell = cell_list[0]
    else:
        decoder_cell = tf.contrib.rnn.MultiRNNCell(cell_list)
    
    # Helper
    
    # attention
    attention_mechanism = tf.contrib.seq2seq.LuongAttention(
        attention_hidden_size, encoder_outputs,
        memory_sequence_length=x_real_len,scale=True)
    decoder_cell = tf.contrib.seq2seq.AttentionWrapper(
        decoder_cell, attention_mechanism,
        attention_layer_size=attention_output_size)
    
    
    projection_layer = layers_core.Dense(
        target_vocat_size, use_bias=False)
    
    
    
    # Dynamic decoding
    with tf.variable_scope("decode_layer"):
        helper = tf.contrib.seq2seq.TrainingHelper(
            decoder_emb_inp,sequence_length= y_len)
        decoder = tf.contrib.seq2seq.BasicDecoder(
            decoder_cell, helper, initial_state = decoder_cell.zero_state(dtype=tf.float32,batch_size=batch_size),
            output_layer=projection_layer)
       
        outputs, _,___  = tf.contrib.seq2seq.dynamic_decode(decoder)
        logits = outputs.rnn_output

        target_weights = tf.sequence_mask(
            y_real_len, seq_max_len, dtype=logits.dtype)
    
    # predicting
    # Helper
    with tf.variable_scope("decode_layer", reuse=True):
        helper_predict = tf.contrib.seq2seq.GreedyEmbeddingHelper(
            embedding_decoder,
            tf.fill([batch_size], ch2ind['<go>']), 0)
        decoder_predict = tf.contrib.seq2seq.BasicDecoder(
            decoder_cell, helper_predict, initial_state = decoder_cell.zero_state(dtype=tf.float32,batch_size=batch_size),
            output_layer=projection_layer)
        outputs_predict,_, __ = tf.contrib.seq2seq.dynamic_decode(
            decoder_predict, maximum_iterations=test_y.shape[1] * 2)
    translations = outputs_predict.sample_id

    # calculate loss
    crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=y, logits=logits)
    train_loss = (tf.reduce_sum(crossent * target_weights) /
        batch_size)
    
    optimizer_ori = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    trainable_params = tf.trainable_variables()
    gradients = tf.gradients(train_loss, trainable_params)
    clip_gradients, _ = tf.clip_by_global_norm(gradients, max_grad)
    global_step = tf.Variable(0, trainable=False, name='global_step')
    optimizer = optimizer_ori.apply_gradients(
            zip(clip_gradients, trainable_params), global_step=global_step)
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(train_loss)
    #trainop = tflearn.TrainOp(loss=train_loss, optimizer=optimizer,
    #                          metric=train_loss, batch_size=64)


In [25]:
def cal_acc(logits,target):
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target[:,:seq_max_len], logits[:,:seq_max_len]))


# init the model

In [27]:
session.run(tf.global_variables_initializer())

In [26]:
saver = tf.train.Saver()

In [27]:
saver.restore(session,'middleresult/align_word_deep/result_2_27796')

INFO:tensorflow:Restoring parameters from middleresult/align_word_deep/result_2_27796


In [31]:
saver.save(session,'middleresult/result_word_deep')

'middleresult/result_word_deep'

In [28]:
from utils import Dataset,ProgressBar

In [29]:
from utils import *
train_set = Dataset(train_x,train_y)
test_set = Dataset(test_x,test_y)

In [30]:
def get_bleu_score(predict,target):
    try:
        target = [[[j for index,j in enumerate(i) if j > 0 or index < 4]] for i in target]
        predict = [[j for index,j in enumerate(i) if j > 0 or index < 4] for i in predict]
        BLEUscore = nltk.translate.bleu_score.corpus_bleu(target,predict)
    except:
        BLEUscore = -1
    return BLEUscore

In [31]:
import numpy as np
def calc_test_loss(test_set = Dataset(test_x,test_y),display=True):
    accs = []
    worksum = int(len(test_x) / batch_size)
    loss_list = []
    predict_list = []
    target_list = []
    source_list = []
    pb = ProgressBar(worksum=worksum,info="validating...",auto_display=display)
    pb.startjob()
    #test_set = Dataset(test_x,test_y)
    for j in range(worksum):
        batch_x,batch_y = test_set.next_batch(batch_size)
        lx = [seq_max_len] * batch_size
        ly = [seq_max_len] * batch_size
        bx = [np.sum(m > 0) for m in batch_x]
        by = [np.sum(m > 0) for m in batch_y]
        tmp_loss,tran = session.run([train_loss,translations],feed_dict={x:batch_x,y:batch_y,
                                                     y_in:
                                                     np.concatenate((
                                                     np.ones((batch_y.shape[0],1),dtype=np.int) * ch2ind['<go>'],batch_y[:,:-1]) ,axis=1)
                                                     ,x_len:lx,y_len:ly,
                                                                        y_real_len:by,
                                                                        x_real_len:bx})
        loss_list.append(tmp_loss)
        tmp_acc = cal_acc(tran,batch_y)
        accs.append(tmp_acc)
        predict_list += [i for i in tran]
        target_list += [i for i in batch_y]
        source_list += [i for i in batch_x]
        pb.complete(1)
    return np.average(loss_list),np.average(accs),get_bleu_score(predict_list,target_list),predict_list,target_list,source_list

# calculate the initial loss and see what model outputs in the begining.

In [36]:
w_loss,w_acc,bleu_score,predict_list,target_list,source_list = calc_test_loss(Dataset(train_x[::100],train_y[::100]))

validating... 100.00 % [==================================================>] 280/280 	 used:322s eta:0 s

In [37]:
w_loss,w_acc,bleu_score

(172.89665, 5.5803571428571429e-06, 0.09401507732715986)

In [39]:
def get_all_text(x):
    return [' '.join([ind2ch.get(j,'') for j in i]) for i in x]
def get_all_en_text(x):
    return [' '.join([ind2en.get(j,'') for j in i]) for i in x]

In [39]:
texts = get_all_text(predict_list)
texts[:10]

['soc soc soc soc soc soc 村 用刀 村 换气 换气 主体 主体 源岩 主体 三类 带个 三类 干涸 特使 毛病 毛病 毛病 一分钟 条码 条码 条码 条码 小牛 条码 条码 条码 条码 条码 条码 条码 条码 条码 条码 条码 条码 条码 条码 条码 成团 成团 成团 成团 成团 袜子 袜子 袜子 袜子 变质 成团 变质 变质 变质 浇灌 浇灌 浇灌 浇灌 浇灌 浇灌 浇灌 浇灌 浇灌 浇灌 浇灌 浇灌 居住者 苹果酸 苹果酸 苹果酸 浇灌 苹果酸 苹果酸 苹果酸 苹果酸 血糖',
 '里夫 里夫 里夫 里夫 量度 通达 通达 通达 通达 通达 通达 显微镜 显微镜 再要 再要 再要 翻滚 翻滚 翻滚 翻滚 翻滚 翻滚 翻滚 扭伤 吸干 光彩照人 光彩照人 光彩照人 宣告破产 翻倒 翻倒 宣告破产 宣告破产 宣告破产 宣告破产 热电厂 热电厂 凸出 软盘 软盘 ４ ４ 软盘 发达国家 软盘 发达国家 榜 icp 无线 罗素 罗素 罗素 罗素 罗素 罗素 罗素 罗素 罗素 罗素 罗素 罗素 做错事 做错事 做错事 做错事 初期 做错事 初期 初期 做错事 初期 初期 初期 初期 初期 初期 初期 主程序 主程序 主程序',
 '复合板 复合板 复合板 复合板 复合板 复合板 复合板 复合板 复合板 复合板 复合板 复合板 平和 平和 最深处 最深处 复合板 穿 六点钟 穿 剪刀 穿 穿 穿 穿 冻干 冻干 冻干 冻干 冻干 冻干 冻干 冻干 冻干 冻干 冻干 冻干 冻干 冻干 冻干 冻干 冻干 冻干 冻干 成团 成团 成团 成团 成团 成团 成团 成团 成团 成团 成团 成团 成团 成团 成团 成团 成团 成团 成团 二维 公共交通 公共交通 公共交通 公共交通 品性 品性 复合板 复合板 复合板 确 复合板 公共交通 确 英亩 英亩 英亩',
 '分租 分租 分租 厚厚 一定量 令人讨厌 令人讨厌 令人讨厌 令人讨厌 速冻 探讨 探讨 探讨 探讨 探讨 探讨 探讨 探讨 探讨 探讨 分子筛 探讨 探讨 分子筛 分子筛 山区 质地 分子筛 质地 分子筛 源代码 烟火 烟火 烟火 烟火 烟火 烟火 烟火 原虫 圆柱齿轮 原虫 黑暗 黑暗 黑暗 登入 登入 圆珠笔 圆珠笔 圆珠笔 崖 星球大战 虚无 虚无 虚无 虚无 虚无 虚无 虚无 星球大战 

In [40]:
texts = get_all_text(target_list)
texts[:10]

['你 不 去 找 格兰特 博士 吗 ?                                ',
 '巴德 和 其他 地方 的 成员 正在 意识 到 这 一点 即 你 能否 把 学生 在 从业 前 吸引 过来 ， 你 就 必须 能 把 他们 的 职业 与 科学 挂钩 。      ',
 '呃 ， <unk> 年 ， 爱迪生 发明 了 断路器 。                              ',
 '我 保证 付款 将 不 <unk> 六月 末                                ',
 '<unk> 民族 机床 工业 展示 国际 先进 水平 — — “ <unk> & <unk> 2006 ” 新闻 发布会                      ',
 '<unk> someone out of something 指 的 是 ‘ 说服 某人 不要 做 某事 / 让 他们 改变 主意 ’ 。                   ',
 '毕业 后 许多 学生 兴高采烈 。                                  ',
 '安徽 地区 <unk> 的 研究 与 展望                                 ',
 '某 一位 舞者 会 超越 当下 ， 超然 出世                               ',
 '( b ) 该 遗嘱 所作 的 其他 处置 亦须 生效 ， 除非 该 遗嘱 内 显示 出 立遗嘱 人 意欲 该项 处置 因 该段 婚姻 而 撤销 。           ']

# now train the model

In [35]:
#tran.shape
i_save = 1
j_save = 27798

In [36]:
print(i_save,j_save)

1 27798


In [37]:
model_path = 'align_word_deep'

In [44]:
os.mkdir('middleresult/{}'.format(model_path))
os.mkdir('eval/{}'.format(model_path))

In [40]:
n_epoch = 15
restore = True
lr = 1
for i in range(i_save,n_epoch):
    
    i_save = i
    worksum = int(len(train_y)/batch_size)
    pb = ProgressBar(worksum=worksum)
    pb.startjob()
    train_loss_list = []
    train_acc_list = []
    for j in range(worksum):
        if restore == True and j < j_save:
            pb.finishsum += 1
            continue
        restore = False
        
        j_save = j
        batch_x,batch_y = train_set.next_batch(batch_size)
        lx = [seq_max_len] * batch_size
        ly = [seq_max_len] * batch_size
        bx = [np.sum(m > 0) for m in batch_x]
        by = [np.sum(m > 0) for m in batch_y]
        by =[m + 2  if m < seq_max_len - 1 else m for m in by ]
        _, loss = session.run([optimizer,train_loss],feed_dict={x:batch_x,y:batch_y,x_len:lx,y_len:ly,learning_rate:lr,y_in:
                                                                np.concatenate((
                                                                np.ones((batch_y.shape[0],1),dtype=np.int) * ch2ind['<go>'],batch_y[:,:-1]) ,axis=1)
                                                                ,y_real_len:by,
                                                                x_real_len:bx
                                                               })
        train_loss_list.append(loss)
        #tmp_train_acc = cal_acc(tran,batch_y)
        #train_acc_list.append(tmp_train_acc)
        pb.info = "iter {} loss:{} lr:{}".format(i + 1,loss,lr)
        val_step = int(worksum / 4)
        if j % val_step == 0 and j != 0:
            test_loss,test_acc,bleu_score,predict_list,target_list,source_list = calc_test_loss()
            _,train_acc,train_bleu_score,train_predict_list,train_target_list,train_source_list = calc_test_loss(Dataset(train_x[::100],train_y[::100]),display=False)
            predict_texts = get_all_text(predict_list)
            target_texts = get_all_text(target_list)
            source_texts = get_all_en_text(source_list)
            
            train_predict_texts = get_all_text(train_predict_list)
            train_target_texts = get_all_text(train_target_list)
            train_source_texts = get_all_en_text(train_source_list)
            
            with open('eval/{}/{}_{}_predict'.format(model_path,i + 1,j),'w',encoding='utf-8') as whdl:
                for line in predict_texts:
                    whdl.write("{}\n".format(line))
            with open('eval/{}/{}_{}_target'.format(model_path,i + 1,j),'w',encoding='utf-8') as whdl:
                for line in target_texts:
                    whdl.write("{}\n".format(line))
            with open('eval/{}/{}_{}_source'.format(model_path,i + 1,j),'w',encoding='utf-8') as whdl:
                for line in source_texts:
                    whdl.write("{}\n".format(line))
                    
            with open('eval/{}/{}_{}_predict_train'.format(model_path,i + 1,j),'w',encoding='utf-8') as whdl:
                for line in train_predict_texts:
                    whdl.write("{}\n".format(line))
            with open('eval/{}/{}_{}_target_train'.format(model_path,i + 1,j),'w',encoding='utf-8') as whdl:
                for line in train_target_texts:
                    whdl.write("{}\n".format(line))
            with open('eval/{}/{}_{}_source_train'.format(model_path,i + 1,j),'w',encoding='utf-8') as whdl:
                for line in train_source_texts:
                    whdl.write("{}\n".format(line))
            print("\niter {} step {} train loss {} train acc {} test loss {} test acc {} bleu {} lr {}\n".format(i+1,j,np.average(train_loss_list[-val_step:]),train_acc,test_loss,test_acc,bleu_score,lr))
            try:
                saver = tf.train.Saver()
                saver.save(session,'middleresult/{}/result_{}_{}'.format(model_path,i + 1,j))
            except:
                print('save fail')
        lr_step = int(worksum / 2) - 1
        if j % lr_step == 0 and j != 0:
            if (i + 1) >= 7:
                lr = lr / 2
        pb.complete(1)

validating... 100.00 % [==================================================>] 280/280 	 used:265s eta:0 s
iter 3 step 6949 train loss 62.169124603271484 train acc 0.6218178013392858 test loss 59.426414489746094 test acc 0.6189327566964287 bleu 0.10504980754240471 lr 1

validating... 100.00 % [==================================================>] 280/280 	 used:403s eta:0 s
iter 3 step 13898 train loss 57.09894561767578 train acc 0.5874288504464285 test loss 57.24261474609375 test acc 0.5840652901785715 bleu 0.07951183898782706 lr 1

validating... 100.00 % [==================================================>] 280/280 	 used:320s eta:0 s
iter 3 step 20847 train loss 56.06503677368164 train acc 0.5329617745535714 test loss 56.29206085205078 test acc 0.5302497209821428 bleu 0.053528670373905136 lr 1

validating... 100.00 % [==================================================>] 280/280 	 used:260s eta:0 s
iter 3 step 27796 train loss 55.227806091308594 train acc 0.6143987165178572 test loss 55

ResourceExhaustedError: OOM when allocating tensor with shape[1024,50003]
	 [[Node: gradients/decode_layer/decoder/while/BasicDecoderStep/dense/MatMul_grad/MatMul_1 = MatMul[T=DT_FLOAT, transpose_a=true, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](gradients/decode_layer/decoder/while/BasicDecoderStep/dense/MatMul_grad/MatMul_1/StackPop, gradients/decode_layer/decoder/while/TensorArrayWrite/TensorArrayWriteV3_grad/TensorArrayReadV3)]]
	 [[Node: gradients/decode_layer/decoder/while/BasicDecoderStep/TrainingHelperNextInputs/cond/Merge_grad/cond_grad/_167 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_1529_gradients/decode_layer/decoder/while/BasicDecoderStep/TrainingHelperNextInputs/cond/Merge_grad/cond_grad", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"](^_cloopgradients/decode_layer/decoder/while/BasicDe

In [50]:
session

In [51]:
len([ind2en.get(i,'') for i in test_x[1]])

40

# try to translate

In [227]:
sent = 'you are too stupid to know that you are an idiot .'.split()

In [228]:
sent

['you',
 'are',
 'too',
 'stupid',
 'to',
 'know',
 'that',
 'you',
 'are',
 'an',
 'idiot',
 '.']

In [229]:
sents = [en2ind.get(i) for i in sent]

In [230]:
sents = tf.contrib.keras.preprocessing.sequence.pad_sequences([sents],seq_max_len,padding='post')

In [231]:
sents

array([[  13,   31,  198, 3197,    7,   83,   17,   13,   31,   35, 7122,
           3,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0]])

In [232]:
np.repeat(sents,35,axis=0)

array([[ 13,  31, 198, ...,   0,   0,   0],
       [ 13,  31, 198, ...,   0,   0,   0],
       [ 13,  31, 198, ...,   0,   0,   0],
       ..., 
       [ 13,  31, 198, ...,   0,   0,   0],
       [ 13,  31, 198, ...,   0,   0,   0],
       [ 13,  31, 198, ...,   0,   0,   0]])

In [233]:
sum(sents[0] > 0) + 1

13

In [234]:
tran = session.run([translations],feed_dict={x:np.repeat(sents,64,axis=0),x_len:[35] * 64, x_real_len:[sum(sents[0] > 0) + 1] * 64})

In [235]:
from collections import Counter
Counter(''.join([ind2ch.get(i,'') for i in j]) for j in tran[0]).most_common(5)

[('你太傻了，知道你是个白痴', 34),
 ('你太傻了，不知道你是个白痴', 8),
 ('你太傻了，你就知道你是个白痴', 6),
 ('你太愚蠢了，知道你是个白痴', 2),
 ('你太傻了，知道你是白痴', 2)]

In [203]:
en2ind['<go>'],ch2ind['<go>']

(50002, 8824)

# release the model

In [135]:
!mkdir release

A subdirectory or file release already exists.


In [136]:
os.mkdir('release/align_and_translate_char_50000')

In [137]:
saver.save(session,'release/align_and_translate_char_50000/align_and_translate_model')

'release/align_and_translate_char_50000/align_and_translate_model'

In [207]:
with open('release/align_and_translate_char_50000/dic.pkl','wb') as whdl:
    pickle.dump(
        ( 
            ind2ch,
            ch2ind,
            ind2en,
            en2ind,
        ),whdl,protocol=2)